###Cross-encoders concatenate both inputs into a single sequence and process them jointly through a transformer, allowing full token-to-token attention for richer interactions. They output a direct relevance score but require processing each pair anew, making them slower.

###Why Reranking?
We use reranking because initial retrieval with embeddings is fast but not very precise. Reranking with a cross-encoder improves relevance by jointly analyzing the query and documents, which leads to more accurate results for the final LLM response.

###Can you give an example?

You: Sure. Query: "How does reranking improve RAG?" A bi-encoder might rank a doc about "RAG music festivals" high because of keyword overlap, while burying the perfect technical explanation. Reranking uses cross-encoders to score query-document pairs deeply, promoting truly relevant docs to top-5 for the LLM.

###What are the benefits?

You: 10-30% hit rate improvement, noise reduction, and better LLM context. Retrieval is RAG's biggest bottleneck—reranking fixes that without sacrificing scale, since you only rerank a small candidate set.

In [ ]:
!pip install -U langchain langchain_core langchain_community

In [ ]:
!pip install langchain_google_genai
!pip install langchain-text-splitter
!pip install pincone
!pip install pypdf

In [ ]:
!pip install langchain sentence-transformers faiss-cpu
!pip install pypdf

In [ ]:
!pip install langchain-text-splitters

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS


In [ ]:
loader=TextLoader("mynta_faq.txt")# go to the file and geting ready to read it but it is not reading yet
docs=loader.load()#now it is reading the file

In [ ]:
splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
docs=splitter.split_documents(docs)


In [ ]:
docs[:2]

In [ ]:
import getpass
import os
os.environ["GOOGLE_API_KEY"]=getpass.getpass("enter api key")

In [ ]:
#embedings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",
    task_type="retrieval_document",
    output_dimensionality=512
)



In [ ]:
db=FAISS.from_documents(docs,embeddings)

In [ ]:
list(db.docstore._dict.values())[:2]

In [ ]:
retriver=db.as_retriever(search_kwargs={"k":10})

###Cross-Encoder Reranker


In [ ]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")


In [ ]:
def rerank(query,docs):
  pairs=[[query,doc.page_content] for doc in docs ]
  scores=cross_encoder.predict(pairs)

  scored_docs=list(zip(docs,scores))
  scored_docs=sorted(scored_docs,key=lambda x:x[1],reverse=True)

  return [doc for doc, score in scored_docs[:3]] #top 3 retrival

In [ ]:
query="Why are there different prices for the same product? Is it legal?"
# step1: retrval using the bi encoders
retrived_docs=retriver.invoke(query)

#step2: Reranking using cross encoders

top_docs= rerank(query,retrived_docs)

for i , doc in enumerate(top_docs):
  print(f"\n Rank{i+1}:\n",doc.page_content)